In [1]:
"""import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from  av.io import read
import pandas as pd
import datetime
import tensorflow as tf
#tf.enable_eager_execution()
"""

from glob import glob
from sklearn.preprocessing import normalize
from tensorflow import keras

import string
import os

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from  av.io import read
import pandas as pd
import datetime
import tensorflow as tf
from tensorflow.contrib import lite
#tf.enable_eager_execution()

from glob import glob

from sklearn.preprocessing import normalize
from tensorflow import keras
from tensorflow.keras import backend as K

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

from tensorflow.python.platform import gfile

In [2]:
## Config, changes here have to be applied to Android-App too

use_rotation_sensor = False
use_accelerometer_sensor = True
use_gyroscope_sensor = True
use_magnetic_sensor = False
sensor_inputs = 0
normalize_vector = np.array([])

if use_rotation_sensor:
    sensor_inputs += 5
    normalize_vector = np.append(normalize_vector, [1,1,1,1,1], axis=0)

if use_accelerometer_sensor:
    sensor_inputs += 3
    normalize_vector = np.append(normalize_vector, [15,15,15], axis=0)
    
if use_gyroscope_sensor:
    sensor_inputs += 3
    normalize_vector = np.append(normalize_vector, [3,3,3], axis=0)

if use_magnetic_sensor:
    sensor_inputs += 3
    normalize_vector = np.append(normalize_vector, [40,40,40], axis=0)


cut_shape = (100, sensor_inputs)
gestures = {0: 'noise', 1: 'Left', 2: 'Right'}
gestures_reverse = {"left" : 1, "noise" : 0, "right" : 2}

data_folder = "data/"
patterns = {"left" : "swipe_left_20*.mkv", "right" : "swipe_right_20*.mkv", "noise" : "*noise*.mkv"} 
data_split_folder = "data_singled/"

# Definition of Data-Handling Functions

In [3]:
def read_file(filename, lable=None):
    data, meta = read(" ".join(["a:" + str(i) for i in range(4)]), filename, 50)
    datapairs = list(zip(data, meta))
    
    df = pd.DataFrame()
    names = []
    col_name_tpls = []
    for sensor_data, stream_object in datapairs:
        meta_dict = stream_object.metadata
        # print(meta_dict)
        name = meta_dict["NAME"]
        names.append(name)
        n_cols = len(sensor_data)
        col_names = []
        for i in range(n_cols):
            col_name = name + "_" + str(i)
            col_names.append(col_name)
            df[col_name] = pd.Series(sensor_data.T[:,i])
        col_name_tpls.append(col_names)
    df.index = pd.TimedeltaIndex(freq="20ms", start=0.0, periods=len(df))
    return df, col_name_tpls

def trim_data(data_frame, start=0.0, end=None):
    if end:
        df = data_frame[pd.Timedelta(10**9 * start):pd.Timedelta(10**9 * end)]
    else:
        df = data_frame[pd.Timedelta(10**9 * start):]
    return df

def cut_data(data_frame, cuts):
    df_cut_list = np.ndarray(shape=(len(cuts),) + cut_shape)
    for i, cut in enumerate(cuts):
        df_i = data_frame[pd.Timedelta(10**9 * cut[0]):pd.Timedelta(10**9 * cut[1])]
        start_chanel = 0
        if len(df_i) < 100:
            continue
        if not use_rotation_sensor:
            start_chanel = 5
        df_cut_list[i, : , :] = df_i.iloc[0: 100, start_chanel:8]

    return df_cut_list


def find_peaks(df, sensor, start=0.0, end=None, sel_method="right"):
    peaks = []
    if end:
        df = df[pd.Timedelta(10**9 * start):pd.Timedelta(10**9 * end)]
    else:
        df = df[pd.Timedelta(10**9 * start):]

    for i in df.index:
        value = df[sensor][i]
        if sel_method == "right":
            cond = value > 7.5
        elif sel_method == "left":
            cond = value < -1
        if cond:
            new = True
            for peak in peaks:
                if i - pd.Timedelta(10**9) < peak + pd.Timedelta(10**9):
                    new = False
                    break
            if new:
                peaks.append(i)
    cuts = []
    for peak in peaks:
        t = peak.value / 10 ** 9
        cuts.append((t-1, t+1))
    return cuts 

def normalize_cuts(cuts):
    for i in range(len(cuts)):
        cuts[i, : , :] = cuts[i]/normalize_vector
    return cuts
  

def plot_dataframe(df, col_name_tpls):
    for col_name_tpl in col_name_tpls:
        for col_name in col_name_tpl:
            fig = df[col_name].plot()
            fig.set_title(col_name)
            plt.show()

def plot_cuts(cuts, sensor=None):
    if sensor is None:
        for i, cut in enumerate(cuts):
            plt.plot(cut, label=str(i))
            plt.legend()
            plt.show()
        return
    for cut in cuts:
        cut = cut[sensor]
        cut = cut.values.reshape(-1,1)
        print(cut.shape)
        normed_matrix = normalize(cut, axis=0, norm='l1')
        plt.plot(normed_matrix)
        
        plt.show()

In [4]:
def read_file(filename, lable=None):
    data, meta = read(" ".join(["a:" + str(i) for i in range(4)]), filename, 50)
    datapairs = list(zip(data, meta))
    
    df = pd.DataFrame()
    names = []
    col_name_tpls = []
    for sensor_data, stream_object in datapairs:
        meta_dict = stream_object.metadata
        # print(meta_dict)
        name = meta_dict["NAME"]
        names.append(name)
        n_cols = len(sensor_data)
        col_names = []
        for i in range(n_cols):
            col_name = name + "_" + str(i)
            col_names.append(col_name)
            df[col_name] = pd.Series(sensor_data.T[:,i])
        col_name_tpls.append(col_names)
    df.index = pd.TimedeltaIndex(freq="20ms", start=0.0, periods=len(df))
    return df, col_name_tpls

def trim_data(data_frame, start=0.0, end=None):
    if end:
        df = data_frame[pd.Timedelta(10**9 * start):pd.Timedelta(10**9 * end)]
    else:
        df = data_frame[pd.Timedelta(10**9 * start):]
    return df

def cut_data(data_frame, cuts):
    df_cut_list = np.ndarray(shape=(len(cuts),) + cut_shape)
    for i, cut in enumerate(cuts):
        df_i = data_frame[pd.Timedelta(10**9 * cut[0]):pd.Timedelta(10**9 * cut[1])]
        prev_sensor_channels = 0
        if len(df_i) < 100:
            continue
        if use_rotation_sensor:
            df_cut_list[i, : , 0:5] = df_i.iloc[0: 100, 0:5]
            prev_sensor_channels += 5
        if use_accelerometer_sensor:
            df_cut_list[i, : , prev_sensor_channels:prev_sensor_channels+3] = df_i.iloc[0: 100, 5:8]
            prev_sensor_channels += 3
        if use_gyroscope_sensor:
            df_cut_list[i, : , prev_sensor_channels:prev_sensor_channels+3] = df_i.iloc[0: 100, 8:11]
            prev_sensor_channels += 3
        if use_magnetic_sensor:
            df_cut_list[i, : , prev_sensor_channels:prev_sensor_channels+3] = df_i.iloc[0: 100, 11:14]
            prev_sensor_channels += 3
    return df_cut_list


def find_peaks(df, sensor, start=0.0, end=None):
    peaks = []
    if end:
        df = df[pd.Timedelta(10**9 * start):pd.Timedelta(10**9 * end)]
    else:
        df = df[pd.Timedelta(10**9 * start):]

    for i in df.index:
        value = df[sensor][i]
        if abs(value) > 7:
            new = True
            for peak in peaks:
                if i - pd.Timedelta(10**9) < peak + pd.Timedelta(10**9):
                    new = False
                    break
            if new:
                peaks.append(i)
    cuts = []
    for peak in peaks:
        t = peak.value / 10 ** 9
        cuts.append((t-1, t+1))
    return cuts 

def normalize_cuts(cuts):
    for i in range(len(cuts)):
        cuts[i, : , :] = cuts[i]/normalize_vector
    return cuts
  

def plot_dataframe(df, col_name_tpls):
    for col_name_tpl in col_name_tpls:
        for col_name in col_name_tpl:
            fig = df[col_name].plot()
            fig.set_title(col_name)
            plt.show()

def plot_cuts(cuts, sensor=None):
    if sensor is None:
        for i, cut in enumerate(cuts):
            plt.plot(cut, label=str(i))
            plt.legend()
            plt.show()
        return
    for cut in cuts:
        cut = cut[sensor]
        cut = cut.values.reshape(-1,1)
        print(cut.shape)
        normed_matrix = normalize(cut, axis=0, norm='l1')
        plt.plot(normed_matrix)
        plt.show()

In [5]:
for gesture in patterns.keys():
    gesture_file_list = glob(data_folder + patterns[gesture])
    print(gesture_file_list)

['data/swipe_left_20_0.mkv', 'data/swipe_left_20_1.mkv', 'data/swipe_left_20_2.mkv', 'data/swipe_left_20_3.mkv', 'data/swipe_left_20_4.mkv']
['data/swipe_right_20_0.mkv', 'data/swipe_right_20_1.mkv', 'data/swipe_right_20_2.mkv', 'data/swipe_right_20_3.mkv', 'data/swipe_right_20_4.mkv']
['data/noise_0.mkv', 'data/noise_1.mkv']


In [6]:
def export_data(force_overwrite=False, do_result_plots=False):
    """export all the data!!
    force_overwrite: overwrite already exported data
    do_result_plots: show plots of resulting gesture cuts again.
    """
    for gesture in patterns.keys():
        gesture_file_list = glob(data_folder + patterns[gesture])
        k = 0
        for filename in gesture_file_list:
            assert filename.count("/") == 1
            rev_file = filename[::-1]

            ind = len(filename) - rev_file.index("_")
            gesture_name = filename[filename.index("/")+1:ind-1]
            gesutre_name_copy = gesture_name
            while gesture_name[-1].lower() not in string.ascii_lowercase:
                gesture_name = gesture_name[:-1]
            try:
                num_cuts = int(gesutre_name_copy[len(gesture_name) - len(gesutre_name_copy) +1:])
            except:
                print(f"WARNING, in FILE DOES NOT SET AMOUNT OF CUTS: {filename}")
                num_cuts = False
            number = filename[ind:filename.index(".")]
            if os.path.isfile(data_split_folder + gesture_name + "_" + number + "_000.csv") and not force_overwrite:
                print(f"File {filename} was already exported, force overwrite if needed.")
                continue
            df, col_name_tpls = read_file(filename)
            df["Cywee Accelerometer Sensor_2"].plot()
            plt.show()
            start = int(input("start?"))
            try:
                end = int(input("end?"))
            except:
                end = False
            if end:
                trimed_df = trim_data(df, start, end)
            else:
                trimed_df = trim_data(df, start)
            peaks = find_peaks(trimed_df, "Cywee Accelerometer Sensor_2")
            print(len(peaks), ":", peaks)
            cuts = normalize_cuts(cut_data(trimed_df, peaks))
            print("Found cuts:", len(cuts))
            plot_cuts(cuts)
            r = eval(input("remove cuts?? (divide by \',\') "))
            assert type(r) == list
            l_cuts = []   # list to keep cuts
            for i, cut in enumerate(cuts):
                if i not in r:
                    l_cuts.append(cut)
            #cuts = np.asarray(l)
            if num_cuts:
                if len(l_cuts) != num_cuts:
                    print(f"WARNING: expected {num_cuts} but got {len(l_cuts)}")
                else:
                    print("correct number of cuts reached")
            for n, cut in enumerate(l_cuts):
                n_str = str(n)
                while len(n_str) < 3:
                    n_str = "0" + n_str
                np.savetxt(data_split_folder + gesture_name + "_" + number + "_" + n_str + ".csv", cut, delimiter=",")
            if do_result_plots:
                plot_cuts(l_cuts)

In [7]:
def load_data(patterns=patterns):
    data_dict = {pattern : [] for pattern in patterns}
    for gesture in patterns:
        gesture_file_list = glob(data_split_folder + "/*" + gesture +"*")
        for i, gesture_file in enumerate(gesture_file_list):
            data = np.loadtxt(gesture_file, delimiter=",")
            data_dict[gesture].append(data)
    
    #combine into X and y
    
    X = []
    y = []
    for gesture in patterns:
        X.extend(data_dict[gesture])
        y.extend([gestures_reverse[gesture]] * len(data_dict[gesture]))
    
    
    return np.asarray(X), y

# computer-aided selection of trial windows

### still have to select the correct ones.
#### enter trials to remove as python list with their indices, e.g. [1, 21, 22]

In [8]:
export_data()

File data/swipe_left_20_0.mkv was already exported, force overwrite if needed.
File data/swipe_left_20_1.mkv was already exported, force overwrite if needed.
File data/swipe_left_20_2.mkv was already exported, force overwrite if needed.
File data/swipe_left_20_3.mkv was already exported, force overwrite if needed.
File data/swipe_left_20_4.mkv was already exported, force overwrite if needed.
File data/swipe_right_20_0.mkv was already exported, force overwrite if needed.
File data/swipe_right_20_1.mkv was already exported, force overwrite if needed.
File data/swipe_right_20_2.mkv was already exported, force overwrite if needed.
File data/swipe_right_20_3.mkv was already exported, force overwrite if needed.
File data/swipe_right_20_4.mkv was already exported, force overwrite if needed.
WARNING, in FILE DOES NOT SET AMOUNT OF CUTS: data/noise_0.mkv
File data/noise_0.mkv was already exported, force overwrite if needed.
WARNING, in FILE DOES NOT SET AMOUNT OF CUTS: data/noise_1.mkv
File dat

# Load cleaned data and train keras model, export model as tflite

In [9]:
X, y = load_data()

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)


In [10]:
def  learn_clasifyer(data, lables):
    print(data.shape)
    model = keras.Sequential([
        #keras.layers.Flatten(),
        keras.layers.Flatten(input_shape=cut_shape),
        keras.layers.Dense(100 * len(normalize_vector), activation=tf.nn.relu),
        keras.layers.Dense(3, activation=tf.nn.softmax)
    ])
    model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    model.fit(data, lables, epochs=50)
    test_loss, test_acc = model.evaluate(data, lables)

    print('Test accuracy:', test_acc)
    
    return model

    



In [11]:
tf_model = learn_clasifyer(X_train, y_train)

(193, 100, 6)
Instructions for updating:
Colocations handled automatically by placer.
Epoch 1/50
193/193 [==============================] - 0s 1ms/sample - loss: 0.4998 - acc: 0.8187
Epoch 2/50
193/193 [==============================] - 0s 298us/sample - loss: 0.0589 - acc: 0.9845
Epoch 3/50
193/193 [==============================] - 0s 335us/sample - loss: 0.0239 - acc: 1.0000
Epoch 4/50
193/193 [==============================] - 0s 308us/sample - loss: 0.0081 - acc: 1.0000
Epoch 5/50
193/193 [==============================] - 0s 320us/sample - loss: 0.0042 - acc: 1.0000
Epoch 6/50
193/193 [==============================] - 0s 248us/sample - loss: 0.0029 - acc: 1.0000
Epoch 7/50
193/193 [==============================] - 0s 413us/sample - loss: 0.0023 - acc: 1.0000
Epoch 8/50
193/193 [==============================] - 0s 290us/sample - loss: 0.0019 - acc: 1.0000
Epoch 9/50
193/193 [==============================] - 0s 352us/sample - loss: 0.0015 - acc: 1.0000
Epoch 10/50
193/193 [====

In [13]:
tf_model.evaluate(X_test, y_test)

65/65 [==============================] - 0s 149us/sample - loss: 0.1464 - acc: 0.9538


[0.146413645377386, 0.95384616]

In [12]:
tf_model.save('keras_gesture_model.h5')

converter = lite.TFLiteConverter.from_keras_model_file('keras_gesture_model.h5')
tfmodel = converter.convert()
open ("keras_gesture_model.tflite" , "wb") .write(tfmodel)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


1450684

# After running the notebook:
 - copy the 'keras_gesture_model.tflite' into the res folder of 'CookbookMotion'
 - Recompile and test on the watch!
